<a href="https://colab.research.google.com/github/luifo/IA-image-super-resolution-tensorflow/blob/test/flower_id_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Ruta en googledrive
PATH = "/content/drive/MyDrive/102flowers"
# Ruta de datos de entrada
IN_PATH = PATH + '/jpg001'
# Ruta de datos de salida
OUT_PATH = PATH + '/jpg002'
# Ruta de check points
CK_PATH = PATH + '/checkpints'

# usando codigo de consola listando el directorio de entrada
img_urls = !ls -1 "{IN_PATH}"

# Entrenando con 500 imagenes
n = 500
train_n = round(n*0.80)
# listado randomizado
rand_urls = np.copy(img_urls)

np.random.seed(23) #solo para el tutorial
np.random.shuffle(rand_urls)

# Particion train test}
tr_urls = rand_urls[:train_n]
ts_urls = rand_urls[train_n:n]

print(len(img_urls), len(tr_urls), len(ts_urls))



1 1 0


In [10]:
%tensorflow_version 2.x

In [20]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

# Resize
def resize(inimg, targimg, height, width):
  inimg = tf.image.resize(inimg, [height, width])
  targimg = tf.image.resize(targimg, [height, width])

  return inimg, targimg

# Normalize
def normalize(inimg, targim):
  inimg = (inimg / 127.5) - 1 # 256/2 - 1
  targimg = (targimg / 127.5) - 1 # 256/2 - 1

  return inimg, targimg

#decorador para utilizar la version 1
@tf.function()
# Agregacion de datos random crop + flip
def random_jitter(inimg, targim):

  inimg, targimg = resize(inimg, targimg, 286, 286)
  stacked_image = tf.stack([inimg, targimg], axis=0)
  cropped_image = tf.image.crop(stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

  inimg, targimg = cropped_image[0], cropped_image[1]

  if tf.random.uniform(()) > 0.5:
    inimg = tf.image.flip_left_right(inimg)
    targimg = tf.image.flip_left_right(targimg)

  return inimg, targimg

# Cargar imagenes
def load_image(filename, augment=True):
  # solo cargar los 3primeros canales rgb
  inimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(IN_PATH + '/' + filename)), tf.float32)[..., :3] 
  targimg = tf.cast(tf.image.decode_jpeg(tf.io.read_file(OUT_PATH + '/' + filename)), tf.float32)[..., :3]

  inimg, targimg = resize(inimg, targimg, IMG_HEIGHT, IMG_WIDTH)

  if augment:
    inimg, targimg = random_jitter(inimg, targimg)
  
  inimg, targimg = normalize(inimg, targimg);

  return inimg, targimg

# Entrenando y cargando
def load_train_image(filename):
  return load_image(filename, True)

def load_test_image(filename):
  return load_image(filename, False)

#plt.imshow((load_train_image(rand_urls[0])[0]))
load_image('image_00001.jpg')

UnboundLocalError: ignored

# New Section